# Day 2

In [238]:
from numba import cuda
import numpy as np
import re

In [239]:
colors = {' red': '0', ' green': '1', ' blue': '2'}

In [240]:
with open('input.txt', 'r') as f:
    input = [[[int(re.sub(r' ([a-z]+)', lambda x: colors[x.group()], cubes)) for cubes in hand.split(', ')] for hand in line[:-1].split(': ')[1].split('; ')] for line in f.readlines()]
input

[[[22, 41], [72, 10, 141], [52, 131, 10], [10, 72, 111]],
 [[62, 31], [40, 11, 72], [21]],
 [[42, 30],
  [22, 40, 71],
  [12, 60, 71],
  [51, 102],
  [91, 12, 60],
  [82, 10, 121]],
 [[152, 41, 50], [20, 21, 52], [31, 132], [172, 11, 50]],
 [[111, 40, 32],
  [82, 61],
  [81, 20, 92],
  [40, 162],
  [82, 100, 61],
  [92, 30, 101]],
 [[41, 90, 22], [70, 21, 152], [130, 21, 62], [51, 72, 60], [190, 152, 41]],
 [[122, 50], [51, 62], [50, 152], [52, 50, 51], [11, 112, 20]],
 [[60, 111], [50, 22, 71], [70, 61]],
 [[50, 12, 111], [41, 12], [81, 20], [11, 20, 22], [31, 20]],
 [[72, 40, 111], [131, 10, 12], [72, 61]],
 [[42, 70, 21], [11, 140, 32], [21, 50, 32]],
 [[61, 62, 10], [11, 30, 22], [22, 60, 71]],
 [[60, 101, 132],
  [30, 121, 92],
  [112, 11],
  [40, 32, 131],
  [121, 102, 60],
  [132, 31, 30]],
 [[81, 12, 170],
  [71, 112, 190],
  [190, 92, 21],
  [81, 200, 122],
  [160, 31, 112]],
 [[30, 11, 52], [92, 41], [62, 51, 90]],
 [[132, 10], [22, 21], [11], [102, 80], [40, 31, 92]],
 [[102

In [241]:
lengths_y = [len(i) for i in input]
max_len_y = max(lengths_y)
max_len_y

6

In [242]:
max_len_z = 3

In [243]:
input_padded = np.full((len(input), max_len_y, 3), 0)
input_padded
for i, line in enumerate(input):
    for j, hand in enumerate(line):
        input_padded[i, j, :len(hand)] = hand
input_padded

array([[[ 22,  41,   0],
        [ 72,  10, 141],
        [ 52, 131,  10],
        [ 10,  72, 111],
        [  0,   0,   0],
        [  0,   0,   0]],

       [[ 62,  31,   0],
        [ 40,  11,  72],
        [ 21,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       [[ 42,  30,   0],
        [ 22,  40,  71],
        [ 12,  60,  71],
        [ 51, 102,   0],
        [ 91,  12,  60],
        [ 82,  10, 121]],

       ...,

       [[110,  62, 131],
        [ 42,  20, 121],
        [ 22,  81, 100],
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       [[ 20,  12,   0],
        [ 41,   0,   0],
        [ 71,  12,  10],
        [ 51,  20,   0],
        [ 12,  20,  91],
        [ 21,  30,   0]],

       [[ 70, 112,   0],
        [100,  52,  11],
        [ 70,  11, 132],
        [ 90,   0,   0],
        [ 90, 192,   0],
        [ 90,  92,   0]]])

In [244]:
output = np.ones((len(input_padded), 3), dtype=np.int32)
output

array([[1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1,

In [245]:
@cuda.jit
def count_hands(input_array, output_array):
    x, y, z = cuda.grid(3)
#     print(x,y,z)

    if x < input_array.shape[0] and y < input_array.shape[1] and z < input_array.shape[2]:
        cubes = input_array[x,y,z]
        if cubes == 0:
            return
        color = cubes % 10
        number_of_cubes = cubes // 10
        
        if color == 0 and number_of_cubes <= 12:
            output_array[x, 0] &= 1
        elif color == 0 and number_of_cubes > 12:
            output_array[x, 0] &= 0

            
        if color == 1 and number_of_cubes <= 13:
            output_array[x, 1] &= 1
        elif color == 1 and number_of_cubes > 13:
            output_array[x, 1] &= 0

            
        if color == 2 and number_of_cubes <= 14:
            output_array[x, 2] &= 1
        elif color == 2 and number_of_cubes > 14:
            output_array[x, 2] &= 0
        

In [246]:
threadsperblock = (8,8,8)
blockspergrid_x = np.ceil(len(input_padded) / threadsperblock[0]).astype('int')
blockspergrid_y = np.ceil(max_len_y / threadsperblock[1]).astype('int')
blockspergrid_z = np.ceil(max_len_z / threadsperblock[2]).astype('int')
blockspergrid = (blockspergrid_x, blockspergrid_y, blockspergrid_z)
blockspergrid

(13, 1, 1)

In [247]:
input_padded[2]

array([[ 42,  30,   0],
       [ 22,  40,  71],
       [ 12,  60,  71],
       [ 51, 102,   0],
       [ 91,  12,  60],
       [ 82,  10, 121]])

In [248]:
input_padded[0]

array([[ 22,  41,   0],
       [ 72,  10, 141],
       [ 52, 131,  10],
       [ 10,  72, 111],
       [  0,   0,   0],
       [  0,   0,   0]])

In [249]:
count_hands[blockspergrid, threadsperblock](input_padded, output)

/home/sfroese/envs/aoc23/lib/python3.11/site-packages/numba/cuda/dispatcher.py:538: NumbaPerformanceWarning: Grid size 13 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/home/sfroese/envs/aoc23/lib/python3.11/site-packages/numba/cuda/cudadrv/devicearray.py:886: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.
  warn(NumbaPerformanceWarning(msg))


In [250]:
output

array([[1, 0, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 0],
       [1, 1, 0],
       [0, 1, 0],
       [1, 1, 0],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [0, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [0, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 0, 1],
       [1, 1, 1],
       [0, 1, 1],
       [1, 1, 1],
       [0, 1, 0],
       [1, 1, 1],
       [1, 1, 1],
       [0, 1, 1],
       [1, 0, 1],
       [1, 1, 0],
       [1, 1, 1],
       [1, 1, 1],
       [1, 0, 1],
       [1, 0, 0],
       [1, 1, 1],
       [1, 1, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 1, 1],
       [1, 1, 1],
       [0, 0, 1],
       [0, 1, 1],
       [1, 0, 1],
       [0, 1, 1],
       [0, 0, 1],
       [1, 1, 1],
       [1, 1, 1],
       [0, 1, 1],
       [0, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 0],
       [1, 1, 0],
       [1, 1, 1],
       [1, 1, 1],
       [1, 0, 1],
       [1,

In [251]:
out_red = np.sum(output, axis=1)
valid = np.arange(1, len(out_red)+1)[out_red==3]
valid

array([ 2,  3,  8,  9, 10, 12, 13, 15, 16, 17, 18, 19, 21, 23, 25, 26, 30,
       31, 34, 35, 39, 45, 46, 49, 50, 53, 54, 56, 57, 58, 61, 63, 68, 72,
       73, 75, 76, 77, 78, 80, 81, 83, 87, 88, 89, 90, 94, 98, 99])

In [252]:
np.sum(valid)

2406

# b)

In [253]:
output = np.zeros((len(input_padded), 3), dtype=np.int32)
output

array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0,

In [254]:
@cuda.jit
def count_minimal(input_array, output_array):
    x, y, z = cuda.grid(3)
#     print(x,y,z)

    if x < input_array.shape[0] and y < input_array.shape[1] and z < input_array.shape[2]:
        cubes = input_array[x,y,z]
        if cubes == 0:
            return
        color = cubes % 10
        number_of_cubes = cubes // 10
        
        cuda.atomic.max(output_array, (x, color), number_of_cubes)     

In [255]:
count_minimal[blockspergrid, threadsperblock](input_padded, output)

/home/sfroese/envs/aoc23/lib/python3.11/site-packages/numba/cuda/dispatcher.py:538: NumbaPerformanceWarning: Grid size 13 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/home/sfroese/envs/aoc23/lib/python3.11/site-packages/numba/cuda/cudadrv/devicearray.py:886: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.
  warn(NumbaPerformanceWarning(msg))


In [256]:
output

array([[ 1, 14,  7],
       [ 4,  3,  7],
       [ 6, 12, 10],
       [ 5,  4, 17],
       [10, 11, 16],
       [19,  5, 15],
       [ 5,  5, 15],
       [ 7, 11,  2],
       [ 5, 11,  2],
       [ 4, 13,  7],
       [14,  2,  4],
       [ 6,  7,  6],
       [ 6, 13, 13],
       [20,  8, 12],
       [ 9,  5,  9],
       [ 8,  3, 13],
       [ 5,  6, 10],
       [ 3,  1,  5],
       [ 4,  8,  6],
       [ 3, 16,  1],
       [ 8,  2,  7],
       [13,  7, 13],
       [12, 12,  4],
       [17,  8, 16],
       [ 7,  6, 10],
       [ 8,  8, 13],
       [17,  7,  8],
       [ 7, 18,  8],
       [ 1,  1, 17],
       [ 6,  7, 11],
       [ 1, 10,  9],
       [ 5, 14,  5],
       [ 8, 15, 18],
       [ 2,  5,  5],
       [ 6,  5, 12],
       [18, 14,  4],
       [18, 15,  4],
       [17,  6,  9],
       [12,  2,  1],
       [15, 15,  1],
       [17,  8, 10],
       [10, 18, 11],
       [15,  9, 10],
       [19, 19, 14],
       [10, 13, 10],
       [10, 13,  1],
       [14,  7,  5],
       [17,  

In [257]:
np.sum(np.prod(output, axis=1))

78375